In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [ ]:
import pandas as pd
df_trade = pd.read_csv('results/trade/lags_numeric_log_returns.csv').drop(columns=['Unnamed: 0'])
df_bbo   = pd.read_csv('results/bbo/lags_numeric_log_returns.csv'  ).drop(columns=['Unnamed: 0'])


def preprocess_lags(df):
    df.date = pd.to_datetime(df.date)
    df = df.query(
        "market1 == 'NL' or ((market1 < market2) and market2 != 'NL')")
    df = df[df.lag.abs() < df.lag.abs().quantile(0.99)]
    df['market'] = (df.market1 + ' & ' + df.market2)
    df = df.set_index(['market1', 'market2', 'date']).sort_index()
    return df


df_trade = preprocess_lags(df_trade)
df_bbo = preprocess_lags(df_bbo)


In [ ]:

def massage_df(dataset, markets, frequency, window):
    """Resample, Forward fill and perform rolling average operations"""
    df = df_trade if dataset == 'trade' else df_bbo
    sub_df = df.loc[markets].sort_index()
    threshold = sub_df.lag.abs().quantile(0.975)
    massaged_df = (sub_df.query('lag.abs() < @threshold')
               .resample(frequency)
               .ffill()
               .rolling(window, min_periods=1)
               .mean()
               )
    return massaged_df.reset_index()


def concat_df(dataset, markets, frequency, window):
    """Apply the `massage_df` function to all selected markets"""
    massaged_df = None
    for m in markets:
        a = massage_df(dataset, m, frequency, window)
        a['markets'] = " & ".join(list(m))
        if massaged_df is None:
            massaged_df = a
        else:
            massaged_df = pd.concat((massaged_df, a), axis=0)
    return massaged_df

def plot_corr(dataset, markets, frequency, window):
    """Main interactive viz function"""
    if not window or not frequency or not markets:
        return
   
    fig = px.line(concat_df(dataset, markets, frequency, window), x='date', y='lag',
                  title=f'Rolling Lag for {dataset} data', color='markets')
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    fig.show()

In [ ]:
from ipywidgets import link,interactive, widgets, interact,Layout
config = {
    'layout': Layout(width='80%', height='80px'),
    'style': {'description_width': 'initial'}
}

# Interactive visualisation

<div class="alert alert-block alert-info">
    <b>Tip: use the widgets below to interact with the visualisation</b>
</div>



> **Note**: it may take up to `20 seconds` for the visualisation to appear after the page has loaded. If a problem occurs, just refresh the page and it should work again !

In [ ]:
dataset_selector = widgets.ToggleButtons(
    options=[('trade price', 'trade'), ('mid price',  'bbo')],
    value='trade',
    description='Dataset:',
    disabled=False,
    button_style='info',
    tooltips=['Use trade stock price as feature',
              'Use mid stock price as feature'],
    **config,
)

markets_selector = widgets.ToggleButtons(
    options=[('all', [('NL', 'US'), ('GB', 'US',), ('NL', 'GB')]), ('US & NL', [('NL', 'US')]),
             ('US & GB',  [('GB', 'US')]), ('NL & GB', [('NL', 'GB')])],
    value=[('NL', 'US'), ('GB', 'US',), ('NL', 'GB')],
    description='Markets:',
    disabled=False,
    button_style='info',
    tooltips=['All market pairs : NL, US, GB',
              'Only US & NL data', 'Only US & GB data', 'Only NL & GB data'],
    **config,
)

frequency_selector = widgets.ToggleButtons(
    options=[('daily', 'D'), ('weekly',  'W'), ('monthly', '30d')],
    value='D',
    description='Sample frequency:',
    disabled=False,
    button_style='warning', 
    tooltips=['Daily resampling frequency',
              'Weekly resampling frequency', 'Montly resampling frequency'],
    **config,
)

window_selector = widgets.IntSlider(
    value=10,
    min=1,
    max=30,
    step=1,
    description='Rolling window:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    **config,
)

_ = interact(plot_corr, dataset=dataset_selector, markets=markets_selector, frequency=frequency_selector,
             window=window_selector, )
